In [1]:
import os
import base64
from email.mime.text import MIMEText
from dotenv import load_dotenv
from openai import OpenAI
from langchain_community.tools.gmail.utils import (
    get_gmail_credentials,
)
from googleapiclient.discovery import build

In [ ]:
# Directly assign your OpenRouter API key
api_key = "enter api key or follow readme"

# Optional check
if not api_key:
    raise ValueError("OpenRouter API key is not provided")

# Initialize OpenRouter client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

In [3]:
# Gmail Auth
credentials = get_gmail_credentials(
    token_file="token.json",
    scopes=["https://mail.google.com/"],
    client_secrets_file="credentials.json",
)
service = build("gmail", "v1", credentials=credentials)

In [6]:
# Fetch recent emails
def fetch_recent_emails(max_results=10):
    results = service.users().messages().list(userId='me', labelIds=['INBOX'], maxResults=max_results).execute()
    messages = results.get('messages', [])
    emails = []

    for msg in messages:
        msg_data = service.users().messages().get(userId='me', id=msg['id']).execute()
        headers = msg_data['payload'].get('headers', [])
        subject = next((h['value'] for h in headers if h['name'] == 'Subject'), "(No Subject)")
        sender = next((h['value'] for h in headers if h['name'] == 'From'), "(Unknown Sender)")
        snippet = msg_data.get('snippet', '')
        emails.append({
            "id": msg['id'],
            "subject": subject,
            "sender": sender,
            "snippet": snippet,
        })
    return emails

In [7]:
# Classify email into categories
def classify_email(email):
    prompt = f"""
Classify the following email into one of these categories:
- Personal
- Job Offer
- Job Application Updates
- Newsletter
- Finance
- Promotion
- Other

Email:
From: {email['sender']}
Subject: {email['subject']}
Snippet: {email['snippet']}

Respond with only the category name.
"""
    response = client.chat.completions.create(
        model="openai/gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an email classifier."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=10,
    )
    return response.choices[0].message.content.strip()

In [8]:
# Generate smart replies based on classification
def generate_reply(email, category):
    prompt = f"""
You are a smart and context-aware email assistant.

Generate a short, polite, and contextually appropriate reply for the following email.
Category: {category}

From: {email['sender']}
Subject: {email['subject']}
Content Snippet: {email['snippet']}

Instructions:
- If it's a Job Offer, thank the sender and express interest. Ask about next steps.
- If it's a Job Application Update, acknowledge the update and express continued interest.
- If it's Personal, respond warmly and casually.

Reply:
"""
    response = client.chat.completions.create(
        model="openai/gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful email assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300,
    )
    return response.choices[0].message.content.strip()

In [9]:
# Save the reply as Gmail draft
def create_draft(service, reply_to, subject, message_text):
    message = MIMEText(message_text)
    message['to'] = reply_to
    message['subject'] = f"Re: {subject}"
    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
    draft_body = {'message': {'raw': raw}}

    draft = service.users().drafts().create(userId="me", body=draft_body).execute()
    print(f"📨 Draft created for: {subject}")

In [10]:
# Main execution logic
if __name__ == "__main__":
    emails = fetch_recent_emails()

    for email in emails:
        category = classify_email(email)
        print(f"🔍 {email['subject']} → {category}")
        
        if category in ["Personal", "Job Offer", "Job Application Updates"]:
            reply = generate_reply(email, category)
            create_draft(service, email["sender"], email["subject"], reply)
        else:
            print(f"⏭️ Skipping: {email['subject']} ({category})")

🔍 I want to connect → Personal
📨 Draft created for: I want to connect
🔍 Catching Up Soon? 😊 → Personal
📨 Draft created for: Catching Up Soon? 😊
🔍 Offer Letter for AI/ML Engineer Position at Tata Consultancy Services → Job Offer
📨 Draft created for: Offer Letter for AI/ML Engineer Position at Tata Consultancy Services
🔍 Getting Started with OpenRouter → Newsletter
⏭️ Skipping: Getting Started with OpenRouter (Newsletter)
🔍 BAVYA SREE, review your Google Account settings → Other
⏭️ Skipping: BAVYA SREE, review your Google Account settings (Other)
🔍 Security alert → Other
⏭️ Skipping: Security alert (Other)
🔍 Jio just posted 10 jobs → Newsletter
⏭️ Skipping: Jio just posted 10 jobs (Newsletter)
🔍 Email reply assistant referral link → Other
⏭️ Skipping: Email reply assistant referral link (Other)
🔍 Infosys just posted 30+ jobs → Job Offer
📨 Draft created for: Infosys just posted 30+ jobs
🔍 Build Production-Ready LLMs From Scratch Starting on July 12th! → Newsletter
⏭️ Skipping: Build Produ